# Segmenting and Clustering Neighborhoods in Toronto

### Let's Import The Necessary Libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import folium
import requests
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Extracting The Table Using BeautifulSoup and converting it to Pandas DataFrame

In [2]:
web_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(web_url, 'lxml')
table = soup.find('table', {'class':'wikitable sortable'}).findAll('td')
postcode, borough, neighbourhood = [], [], []
for i in range(0,len(table),3):
    postcode.append(table[i].text.strip())
    borough.append(table[i+1].text.strip())
    neighbourhood.append(table[i+2].text.strip())
df = pd.DataFrame({'PostalCode':postcode, 'Borough':borough,'Neighborhood':neighbourhood})
print(df.head())
df.shape

  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


(289, 3)

### Removing Rows with Borough = "Not Assigned"

In [3]:
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Combining Neighborhoods with same PostalCode and Assigning the value of Borough Whose Neighborhood is "Not assigned"

In [4]:
group = df.groupby('PostalCode')
postcode, borough, neighbourhood = [], [], []
for ind, val in group:
    postcode.append(ind)
    bor = val['Borough'].unique()[0]
    neigh = ', '.join(val['Neighborhood'].unique())
    if neigh=='Not assigned':
        neigh = bor
    borough.append(bor)
    neighbourhood.append(neigh)
df = pd.DataFrame({'PostalCode':postcode, 'Borough':borough,'Neighborhood':neighbourhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Now our DataFrame is ready. Printing the shape of our DataFrame

In [5]:
df.shape

(103, 3)

### Adding the Latitude and Longitude Columns from the provided "Geospatial_Coordinates.csv"

In [6]:
new_df = pd.read_csv('Geospatial_Coordinates.csv')
new_df.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
df = df.merge(new_df, on='PostalCode')

### Finding the latitude and longitude of Toronto

In [7]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Toronto are", latitude, longitude)

The latitude and longitude of Toronto are 43.653963 -79.387207


### Creating the map of Toronto by superimposing the neighborhood on top

In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, long, bor, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = "{}, {}".format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long], 
        radius = 5,
        popup = label, 
        color = 'blue',
        fill = True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html = False
    ).add_to(map_toronto)
map_toronto

### Let us explore the neighborhoods around Scarborough

In [9]:
Scarborough = df[df['Borough']=='Scarborough'][:-1]
Scarborough

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Let's get the geographical cooridnates of Scarborough and plot it's neighborhood

In [10]:
geolocator = Nominatim(user_agent = 'my_application')
location = geolocator.geocode('Scarborough, Toronto')
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Scarborough is", latitude, longitude)

The latitude and longitude of Scarborough is 43.773077 -79.257774


### Visualizing the neighborhoods in Scarborough

In [11]:
scar_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, long, neigh in zip(Scarborough['Latitude'], Scarborough['Longitude'], Scarborough['Neighborhood']):
    label = folium.Popup(neigh, parse_html=True)
    folium.CircleMarker(
        [lat, long], 
        radius = 5,
        popup = label, 
        color = 'blue',
        fill = True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html = False
    ).add_to(scar_map)
scar_map

### Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'XWVGOU1Y5525TKND02OUK3S24AYWJJAHU4LDIPP151NFAXPZ' 
CLIENT_SECRET = '4RDWKT2CX3KSYKBPNOQ4PH2FUOOEOIMKMTK33XMKU0ODA3TM'
VERSION = '20180605' 

### Now, let's create a function to process all the neighborhoods in Scarborough

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
scarborough_venues = getNearbyVenues(names=Scarborough['Neighborhood'],
                                   latitudes=Scarborough['Latitude'],
                                   longitudes=Scarborough['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West


### Let's check the size of the resulting DataFrame and it's first 5 entries

In [15]:
print(scarborough_venues.shape)
scarborough_venues.head()

(83, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


### Let's Check how many values were returned for each type

In [16]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",8,8,8,8,8,8


### Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print("There are {} unique categories".format(len(scarborough_venues['Venue Category'].unique())))

There are 53 unique categories


### Analyze Each Neighborhood

In [18]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,...,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Let's examine the new dataframe size

In [19]:
scarborough_onehot.shape

(83, 54)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,...,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,...,0.5,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000,0.000000
3,Cedarbrae,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.000000,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.142857,0.000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.222222,0.000000,0.0,0.000000,0.222222,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,...,0.0,0.0,0.111111,0.000000,0.00,0.00,0.000000,0.111111,0.000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.0,...,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.166667
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125,0.0,...,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.125,0.000000
9,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000,0.0,...,0.0,0.0,0.166667,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000


### Let's print each Neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2         Skating Rink  0.25
3       Clothing Store  0.25
4  American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4              Pet Store  0.00


----Cedarbrae----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2  Intersec

### Let's put that to a pandas DataFrame

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Metro Station,Park,Coffee Shop,Grocery Store,General Entertainment
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Rental Car Location,General Entertainment,Electronics Store
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bar,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Train Station,Coffee Shop,Hobby Shop,Bus Station,Department Store,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment
9,"Guildwood, Morningside, West Hill",Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint


### Let us cluster the Neighborhoods by running K-means to cluster the neighborhoods into 5 clusters

In [24]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 1, 1, 2, 1, 1, 1])

### Let's create a new dataframe that includes the clusters as well as the top 10 venues for each neighborhood

In [25]:
scarborough_merged = Scarborough

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Indian Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store


### Let's visualize the resulting clusters

In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let us examine each of the clusters

### Cluster 1

In [27]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,0,Pizza Place,Noodle House,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant,Rental Car Location,General Entertainment,Electronics Store


### Cluster 2

In [28]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
2,Scarborough,1,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint
3,Scarborough,1,Coffee Shop,Korean Restaurant,Indian Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store
5,Scarborough,1,Playground,Business Service,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,Scarborough,1,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Metro Station,Park,Coffee Shop,Grocery Store,General Entertainment
8,Scarborough,1,American Restaurant,Motel,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
12,Scarborough,1,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


### Cluster 3

In [29]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Discount Store,Train Station,Coffee Shop,Hobby Shop,Bus Station,Department Store,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment


### Cluster 4

In [30]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Moving Target,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
14,Scarborough,3,Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


### Cluster 5

In [31]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5,scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,4,Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bar,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment


### Observations and Conclusions

We can see from the 5 clusters that the most common place in Scarborough Neighbourhoods are Restaurants, Bar, Park while the 10th Most Common place generally includes of Department Store, Electronic Store or General Entertainment